In [ ]:
# evaluate three RCP
for RCP in ['rcp26','rcp45','rcp85']:
print(RCP)

#at the end of the century (2090)
for ClimateTime in ['2065']:

# using different SLR states (IPCC 6 - SLR assessment)
SLRs = [35,122,221]  # SLR in 2030 SSP126, outer range of low/medium confidence

# assuming differing increases in the total precipitation of a single extreme event resulting from differing climate predictions (--> see 02_max_extremes.ipynb)
prec_increases = [0.9, 1, 1.1]

# assuming differing subdaily precipitation patterns (temporal aggregation / Disaggregation of precipitation signal -> HMT)
for raindissagg in data.filter(regex='Prec').columns:

# evaluating different observed events, for which increased water levels in canals are known
for event in ['2012','2017-03','2018-01','2017-12','2017-02','2011']:

hourly_recharge = data[raindissagg][t1:t2]
tide = data['wl_Knock_Outer'][t1:t2] + slr     # add SLR to tide water level [mm]
hourly_recharge = hourly_recharge.rolling("12h").mean() # smoothen rainfall signal to account for water redistribution within catchment
hourly_recharge *= prec_increase  # scale hourly recharge by change in climate forcing

# evaluate management options
for advance_pump in [0,50]: # lower target water level by 5 cm before incoming event
for Canal_area in [4,6]:    # increase available storage in canals, by increasing water area (polder option in "Freepsumer Meer" and "Großes Meer") -> note, that canal hydraulics are not changed

# evaluate canal flow uncertainty (--> see 01_canal_flow_dh-Q.ipynb)
canal_par = [(1.112 , 4156.), (0.9946, 2142.), (1.045 , 2820.)]

### finally run the model
x['h_store'], q_pump, x['h_min'], x['flow_rec'], pump_cost, x['store'] = drain_cap.storage_model(forcing_data = x,
                                                                                                                                     canal_par = z,
                                                                                                                                     v_store = 0,
                                                                                                                                     h_store_target = -1350,
                                                                                                                                     canal_area = Canal_area,
                                                                                                                                     h_forecast_pump = advance_pump,
                                                                                                                                     h_grad_pump_max = maxdh)